In [1]:
"""Evaluation"""
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
from datetime import datetime
import os.path
import sys
import time

import numpy as np
from six.moves import xrange
import tensorflow as tf

from config import *
from dataset import pascal_voc, kitti
from utils.util import bbox_transform, Timer
from nets import *

data_path = '/rscratch/ruta/squeezeDet/data/KITTI'
image_set = 'val_1000'
eval_dir = '/rscratch/ruta/logs/squeezeDet/8.1/' + image_set
checkpoint_path = '/rscratch/ruta/squeezeDet/data/model_checkpoints/squeezeDet/model.ckpt-87000'
eval_interval_secs = 60
run_once = True

In [2]:
def eval_once(
    saver, ckpt_path, summary_writer, eval_summary_ops, eval_summary_phs, imdb,
    model):
  with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:

    # Restores from checkpoint
    saver.restore(sess, ckpt_path)
    # Assuming model_checkpoint_path looks something like:
    #   /ckpt_dir/model.ckpt-0,
    # extract global_step from it.
    global_step = ckpt_path.split('/')[-1].split('-')[-1]

    num_images = len(imdb.image_idx)

    all_boxes = [[[] for _ in xrange(num_images)]
                 for _ in xrange(imdb.num_classes)]

    _t = {'im_detect': Timer(), 'im_read': Timer(), 'misc': Timer()}

    num_detection = 0.0
    for i in xrange(num_images):
      _t['im_read'].tic()
      images, scales = imdb.read_image_batch(shuffle=False)
      _t['im_read'].toc()

      _t['im_detect'].tic()
      det_boxes, det_probs, det_class = sess.run(
          [model.det_boxes, model.det_probs, model.det_class],
          feed_dict={model.image_input:images})
      _t['im_detect'].toc()

      _t['misc'].tic()
      for j in range(len(det_boxes)): # batch
        # rescale
        det_boxes[j, :, 0::2] /= scales[j][0]
        det_boxes[j, :, 1::2] /= scales[j][1]

        det_bbox, score, det_class = model.filter_prediction(
            det_boxes[j], det_probs[j], det_class[j])

        num_detection += len(det_bbox)
        for c, b, s in zip(det_class, det_bbox, score):
          all_boxes[c][i].append(bbox_transform(b) + [s])
      _t['misc'].toc()

      print ('im_detect: {:d}/{:d} im_read: {:.3f}s '
             'detect: {:.3f}s misc: {:.3f}s'.format(
                i+1, num_images, _t['im_read'].average_time,
                _t['im_detect'].average_time, _t['misc'].average_time))

    print ('Evaluating detections...')
    aps, ap_names = imdb.evaluate_detections(
        eval_dir, global_step, all_boxes)

    print ('Evaluation summary:')
    print ('  Average number of detections per image: {}:'.format(
      num_detection/num_images))
    print ('  Timing:')
    print ('    im_read: {:.3f}s detect: {:.3f}s misc: {:.3f}s'.format(
      _t['im_read'].average_time, _t['im_detect'].average_time,
      _t['misc'].average_time))
    print ('  Average precisions:')

    feed_dict = {}
    for cls, ap in zip(ap_names, aps):
      feed_dict[eval_summary_phs['APs/'+cls]] = ap
      print ('    {}: {:.3f}'.format(cls, ap))

    print ('    Mean average precision: {:.3f}'.format(np.mean(aps)))
    feed_dict[eval_summary_phs['APs/mAP']] = np.mean(aps)
    feed_dict[eval_summary_phs['timing/im_detect']] = \
        _t['im_detect'].average_time
    feed_dict[eval_summary_phs['timing/im_read']] = \
        _t['im_read'].average_time
    feed_dict[eval_summary_phs['timing/post_proc']] = \
        _t['misc'].average_time
    feed_dict[eval_summary_phs['num_det_per_image']] = \
        num_detection/num_images

    print ('Analyzing detections...')
    stats, ims = imdb.do_detection_analysis_in_eval(
        eval_dir, global_step)

    eval_summary_str = sess.run(eval_summary_ops, feed_dict=feed_dict)
    for sum_str in eval_summary_str:
      summary_writer.add_summary(sum_str, global_step)

In [3]:
def evaluate(t, quantize_func):
  """Evaluate."""

  with tf.Graph().as_default() as g:
    mc = kitti_squeezeDet_config()
    mc.BATCH_SIZE = 1 # TODO(bichen): allow batch size > 1
    mc.LOAD_PRETRAINED_MODEL = False
    model = SqueezeDet(mc, t, quantize_func)

    # TODO(ruta): fix this
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
      init = tf.global_variables_initializer()
      sess.run(init)
#       print(sess.run(model.tensor_min))
#       print(sess.run(model.tensor_max))
      i = 0
      for tens in sess.run(model.tensor):
        print(i, tens)
        i+=1
#       i = 0
#       for tens in sess.run(model.tensor_normalized):
#         print(i, tens)
#         i+=1
#       i = 0
#       for tens in sess.run(model.temp1):
#         print(i, tens)
#         i+=1
#       i = 0
#       for tens in sess.run(model.temp2):
#         print(i, tens)
#         i+=1
      i = 0
      for mpq in sess.run(model.tensor_quantized):
        print(i, mpq)
        i+=1
      print(sess.run(model.Wn_temp))
      print(sess.run(model.Wp_temp))
      numpy_tensor = sess.run(model.tensor)
      numpy_tensor_normalized = sess.run(model.tensor_normalized)
      numpy_tensor_quantized = sess.run(model.tensor_quantized)

    imdb = kitti(image_set, data_path, mc)

    # add summary ops and placeholders
    ap_names = []
    for cls in imdb.classes:
      ap_names.append(cls+'_easy')
      ap_names.append(cls+'_medium')
      ap_names.append(cls+'_hard')

    eval_summary_ops = []
    eval_summary_phs = {}
    for ap_name in ap_names:
      ph = tf.placeholder(tf.float32)
      eval_summary_phs['APs/'+ap_name] = ph
      eval_summary_ops.append(tf.summary.scalar('APs/'+ap_name, ph))

    ph = tf.placeholder(tf.float32)
    eval_summary_phs['APs/mAP'] = ph
    eval_summary_ops.append(tf.summary.scalar('APs/mAP', ph))

    ph = tf.placeholder(tf.float32)
    eval_summary_phs['timing/im_detect'] = ph
    eval_summary_ops.append(tf.summary.scalar('timing/im_detect', ph))

    ph = tf.placeholder(tf.float32)
    eval_summary_phs['timing/im_read'] = ph
    eval_summary_ops.append(tf.summary.scalar('timing/im_read', ph))

    ph = tf.placeholder(tf.float32)
    eval_summary_phs['timing/post_proc'] = ph
    eval_summary_ops.append(tf.summary.scalar('timing/post_proc', ph))

    ph = tf.placeholder(tf.float32)
    eval_summary_phs['num_det_per_image'] = ph
    eval_summary_ops.append(tf.summary.scalar('num_det_per_image', ph))

    saver = tf.train.Saver(model.model_params)

    summary_writer = tf.summary.FileWriter(eval_dir, g)
    
    ckpts = set() 

    # When run_once is true, checkpoint_path should point to the exact
    # checkpoint file.
    eval_once(saver, checkpoint_path, summary_writer, eval_summary_ops,
        eval_summary_phs, imdb, model)
    return imdb.correct_percent, numpy_tensor, numpy_tensor_normalized, numpy_tensor_quantized

In [4]:
def minmax(tensor, bias):
    return tf.reduce_min(tensor), tf.reduce_max(tensor)

def std1(tensor, bias):
  if bias:
    mu, var = tf.nn.moments(tensor, [0])
  else:
    mu, var = tf.nn.moments(tensor, [0,1,2,3])
  std = tf.sqrt(var)
  return mu - std, mu + std

def std2(tensor, bias):
  if bias:
    mu, var = tf.nn.moments(tensor, [0])
  else:
    mu, var = tf.nn.moments(tensor, [0,1,2,3])
  std = tf.sqrt(var)
  return mu - 2*std, mu + 2*std

In [5]:
def plot_histogram(tensor, title, xlabel, bins='auto'):
  flat_tensor = tensor.flatten()
  plt.hist(flat_tensor, bins)
  plt.title(title)
  plt.xlabel(xlabel)
  plt.ylabel('Count')
  plt.show()

def plot_percents(title, thresholds, correct_percents, full_precision):
  if full_precision:
    plt.axis([0, 1, 0, 0.6])
    plt.plot([0.579456500986, 0.579456500986], marker='None', linestyle='-',
             color='green', label='Full Precision')
  plt.plot(thresholds, correct_percents, marker='.', linestyle='--',
           color='blue', label='Quantized')
  plt.title(title)
  plt.xlabel('Values of Threshold')
  plt.ylabel('Correct Detection Percentage')
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()

In [6]:
_, numpy_tensor, numpy_tensor_normalized, numpy_tensor_quantized = evaluate(0.9, minmax)

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
0 [[[  5.50680634e-05   9.93640606e-06  -8.15907697e-05 ...,   8.67765775e-05
    -7.36111106e-05   3.54643525e-05]
  [  7.35315698e-05  -1.60251118e-04   1.09425971e-04 ...,  -7.88847756e-05
     1.24849112e-05  -6.43241656e-05]
  [  5.65963674e-05  -2.96171197e-06   6.32552765e-05 ...,   3.44277141e-05
    -1.16310533e-04   9.91639099e-05]
  ..., 
  [ -1.21332654e-04  -1.87226702e-04  -6.93015463e-05 ...,   3.60249032e-05
     1.28648258e-04  -1.25414037e-04]
  [  9.50765843e-06  -1.02589029e-05  -5.56848463e-05 ...,   1.08055086e-04
     5.12375227e-05  -7.04222693e-05]
  [ -7.22504819e-06  -6.47677662e-05  -1.21155072e-05 ...,  -7.70817587e-06
     1.56012451e-04  -1.77609050e-04]]

 [[ -1.24186299e-05  -4.10397442e-05   7.39824536e-05 ...,   6.65355547e-05
     4.85348173e-05  -8.92344106e-05]
  [  1.66454720e-05   1.48049177e-04   5.27827251e-05 ...,   1.44073929e-04
     3.38665377e-05   4.01623256e-05]
  

INFO:tensorflow:Restoring parameters from /rscratch/ruta/squeezeDet/data/model_checkpoints/squeezeDet/model.ckpt-87000
im_detect: 1/3741 im_read: 0.341s detect: 0.217s misc: 0.011s
im_detect: 2/3741 im_read: 0.186s detect: 0.169s misc: 0.011s
im_detect: 3/3741 im_read: 0.134s detect: 0.152s misc: 0.011s
im_detect: 4/3741 im_read: 0.109s detect: 0.143s misc: 0.011s
im_detect: 5/3741 im_read: 0.093s detect: 0.139s misc: 0.011s
im_detect: 6/3741 im_read: 0.083s detect: 0.135s misc: 0.011s
im_detect: 7/3741 im_read: 0.075s detect: 0.133s misc: 0.011s
im_detect: 8/3741 im_read: 0.070s detect: 0.132s misc: 0.011s
im_detect: 9/3741 im_read: 0.065s detect: 0.131s misc: 0.011s
im_detect: 10/3741 im_read: 0.062s detect: 0.129s misc: 0.011s
im_detect: 11/3741 im_read: 0.059s detect: 0.128s misc: 0.011s
im_detect: 12/3741 im_read: 0.057s detect: 0.128s misc: 0.011s
im_detect: 13/3741 im_read: 0.055s detect: 0.127s misc: 0.011s
im_detect: 14/3741 im_read: 0.053s detect: 0.126s misc: 0.011s
im_detec

im_detect: 130/3741 im_read: 0.063s detect: 0.120s misc: 0.011s
im_detect: 131/3741 im_read: 0.063s detect: 0.120s misc: 0.011s
im_detect: 132/3741 im_read: 0.062s detect: 0.120s misc: 0.011s
im_detect: 133/3741 im_read: 0.062s detect: 0.120s misc: 0.011s
im_detect: 134/3741 im_read: 0.062s detect: 0.120s misc: 0.011s
im_detect: 135/3741 im_read: 0.062s detect: 0.120s misc: 0.011s
im_detect: 136/3741 im_read: 0.061s detect: 0.120s misc: 0.011s
im_detect: 137/3741 im_read: 0.061s detect: 0.120s misc: 0.011s
im_detect: 138/3741 im_read: 0.062s detect: 0.120s misc: 0.011s
im_detect: 139/3741 im_read: 0.061s detect: 0.120s misc: 0.011s
im_detect: 140/3741 im_read: 0.061s detect: 0.120s misc: 0.011s
im_detect: 141/3741 im_read: 0.061s detect: 0.120s misc: 0.011s
im_detect: 142/3741 im_read: 0.061s detect: 0.120s misc: 0.011s
im_detect: 143/3741 im_read: 0.061s detect: 0.120s misc: 0.011s
im_detect: 144/3741 im_read: 0.060s detect: 0.120s misc: 0.011s
im_detect: 145/3741 im_read: 0.060s dete

im_detect: 260/3741 im_read: 0.060s detect: 0.119s misc: 0.011s
im_detect: 261/3741 im_read: 0.060s detect: 0.119s misc: 0.011s
im_detect: 262/3741 im_read: 0.060s detect: 0.119s misc: 0.011s
im_detect: 263/3741 im_read: 0.060s detect: 0.119s misc: 0.011s
im_detect: 264/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 265/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 266/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 267/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 268/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 269/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 270/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 271/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 272/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 273/3741 im_read: 0.059s detect: 0.119s misc: 0.011s
im_detect: 274/3741 im_read: 0.058s detect: 0.119s misc: 0.011s
im_detect: 275/3741 im_read: 0.058s dete

KeyboardInterrupt: 

In [ ]:
plot_histogram(numpy_tensor, "Histogram of Kernel Values", "Kernel Values")

In [ ]:
plot_histogram(numpy_tensor_normalized, "Histogram of Normalized Kernel Values", "Normalized Kernel Values")

In [ ]:
def hypertune(thresholds, quantize_func):
  correct_percents = []
  numpy_tensors = []
  for t in thresholds:
    result = evaluate(t, quantize_func)
    correct_percents.append(result[0])
    numpy_tensors.append(result[1:])
  return correct_percents, numpy_tensors

num_points = 1

In [ ]:
minmax_thresholds = [1-(0.1)*i for i in range(num_points)]
minmax_percents, minmax_tensors = hypertune(minmax_thresholds, minmax)

In [ ]:
plot_percents("Full Precision vs. MinMax Quantized", minmax_thresholds, minmax_percents, True)
plot_percents("MinMax Quantized", minmax_thresholds, minmax_percents, False)
print(minmax_percents)

In [ ]:
for i in range(num_points):
  plot_histogram(minmax_tensors[i][2], "Histogram of Minmax Quantized Kernel Values", "Minmax Quantized Kernel Values", 50)

In [ ]:
std1_thresholds = [1-(0.1)*i for i in range(num_points)]
std1_percents, std1_tensors = hypertune(std1_thresholds, std1)

In [ ]:
plot_percents("Full Precision vs. OneStd Quantized", std1_thresholds, std1_percents, True)
plot_percents("OneStd Quantized", std1_thresholds, std1_percents, False)
print(std1_percents)

In [ ]:
for i in range(num_points):
  plot_histogram(std1_tensors[i][2], "Histogram of OneStd Quantized Kernel Values", "OneStd Quantized Kernel Values", 50)

In [ ]:
std2_thresholds = [1-(0.1)*i for i in range(num_points)]
std2_percents, std2_tensors = hypertune(std2_thresholds, std2)

In [ ]:
plot_percents("Full Precision vs. TwoStd Quantized", std2_thresholds, std2_percents, True)
plot_percents("TwoStd Quantized", std2_thresholds, std2_percents, False)
print(std2_percents)

In [ ]:
for i in range(num_points):
  plot_histogram(std2_tensors[i][2], "Histogram of TwoStd Quantized Kernel Values", "TwoStd Quantized Kernel Values", 50)